# Random Forest

**Imports**

In [ ]:
import pandas as pd
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from pickle import dump
from sklearn import tree


**Carga de datos**

In [ ]:
url="https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv"

data=pd.read_csv(url)
data.head()

**Realizamos un EDA completo**

In [ ]:
data.info()

Tenemos 8 variables (todas numéricas) y la variable objetivo (1: diabetes 0: No diabetes), con un total de **768 pacientes**:

* **Pregnancies:** Número de embarazos del paciente (numérico)
* **Glucose:** Concentración de glucosa en plasma a las 2 horas de un test de tolerancia oral a la glucosa (numérico)
* **BloodPressure:** Presión arterial diastólica (medida en mm Hg) (numérico)
* **SkinThickness:** Grosor del pliegue cutáneo del tríceps (medida en mm) (numérico)
* **Insulin:** Insulina sérica de 2 horas (medida en mu U/ml) (numérico)
* **BMI:** Índice de masa corporal (numérico)
* **DiabetesPedigreeFunction:** Función de pedigrí de diabetes (numérico)
* **Age:** Edad del paciente (numérico)
* **Outcome:** Variable **objetivo** de clase (0 o 1), siendo 0 negativo en diabetes y 1, positivo (numérico)

Vamos a buscar y eliminar los duplicados y los nulos: No existen duplicados ni nulos.

In [ ]:
print(data.duplicated().sum())
print(data.isnull().sum())

Exploramos las variables, distribuciones y datos atípicos:

In [ ]:
j=0
fig, axis= plt.subplots(2,4,figsize=(10,4))
for i in data.columns.drop("Outcome"):

    axis[0,j].plot(data[i])
    sns.boxplot(data=data,x=i,ax=axis[1,j])
    j+=1
    if j==4: 
        j=0
        plt.tight_layout()
        plt.show()
        if i!=data.columns.drop("Outcome")[len(data.columns)-2]:
            fig, axis= plt.subplots(2,4,figsize=(10,4))


Limpieza de datos atipicos: A simple vista, vemos algunos datos atípicos que probablemente sean errores en la medición: Glucosa 0, BMI 0, Blood Pressure 0. 
Vamos a ver si son los mismos pacientes para eliminarlos directamente, sino esos valores erróneos los llevaremos a la media.

También vamos a limpiar los datos de atípicos, intentando no disminuir demasiado el dataset ya que la cantidad de datos no es muy elevada:

In [ ]:
Gl0=data[data["Glucose"]==0].index
BMI0=data[data["BMI"]==0].index
BP0=data[data["BloodPressure"]==0].index

union0=[x for x in Gl0 if x in BMI0 and x in BP0]

print(f"El número de pacientes con 0 en Glucosa es: {len(Gl0)}\nEl número de pacientes con 0 en BMI es:{len(BMI0)}\nEl número de pacientes con 0 en BP0 es: {len(BP0)} \nEl número de pacientes con todos los indicadores en 0 es: {len(union0)} ")

Hemos comprobado que los valores nulos en las variables Glucose, BMI y BloodPressure no coinciden en los tres casos, por tanto, alguna de las variables sí se está tomando correctamente en esos casos. Nos llevamos esos atípicos, junto con el resto a la media.

In [ ]:
#Definimos una función para llevar los atípicos a la media:

def out_mean(var,inf=5,sup=95):
    
    lim_inf = np.percentile(var, inf)
    lim_sup = np.percentile(var, sup)
    outliers = (var < lim_inf) | (var > lim_sup)
    var_clean=var.copy()
    var_clean[outliers]=np.mean(var[~outliers])
    return var_clean    


Vamos a limpiar todas las variables:

In [ ]:
data_clean=pd.DataFrame()
for i in data.columns.drop("Outcome"):
    data_clean[i]=out_mean(data[i])
    fig, axis = plt.subplots(1,2,figsize=(10,4))
    sns.boxplot(x=data[i],ax=axis[0])
    x=np.percentile(data[i],75)
    sns.boxplot(x=data_clean[i],ax=axis[1])
    axis[0].text(x,0.7,round(data[i].describe(),2),fontsize=10, ha='right', va='top')
    axis[1].text(x,0.7,round(data_clean[i].describe(),2),fontsize=10, ha='right', va='top')
data_clean["Outcome"]=data["Outcome"]

Comprobamos la correlación con la variable objetivo del dataset limpio y con atípicos:

In [ ]:
j=0
col="blue"
fig, axis= plt.subplots(1,4,figsize=(10,4))
for i in data.columns.drop("Outcome"):

    sns.regplot(data=data,x=i,y="Outcome",ax=axis[j],color=col)
    sns.regplot(data=data_clean,x=i,y="Outcome",ax=axis[j+1],color=col)
    x1=np.percentile(data[i],85)
    x2=np.percentile(data_clean[i],85)
    axis[j].text(x1,0.8,"corr=" + str(round(data[i].corr(data["Outcome"]),2)),fontsize=10, ha='right', va='top')
    axis[j+1].text(x2,0.8,"corr=" + str(round(data_clean[i].corr(data_clean["Outcome"]),2)),fontsize=10, ha='right', va='top')
    j+=2
    col="red"
    if j==4: 
        j=0
        col="blue"
        plt.tight_layout()
        plt.show()
        if i!=data.columns.drop("Outcome")[len(data.columns)-2]:
            fig, axis= plt.subplots(1,4,figsize=(10,4))

Como podemos ver, con la limpieza hemos disminuido la amplitud del intervalo de confianza en todos los casos.

Si quisieramos disminuir el numero de caracteristicas podríamos empezar con las variables: Insulin y Skinthickness que tienen una correlación baja con la variable objetivo. Más adelante probaremos a ver si mejora el modelo.

Como estamos con un random forest, no vamos a normalizar las variables.

Realizamos un parallel coordinates como paso final para visualizar todas las variables predictoras frente a la variable objetivo:

In [ ]:
pd.plotting.parallel_coordinates(data_clean, "Outcome", color = ("#E58139", "#39E581"))
plt.xticks(rotation=45)



Vemos cómo la variable **Glucose** es la que más impacto tiene y la variable **Age** también puede tener impacto. Junto con las regresiones concluimos que esas variables y **BMI** son las que **mayor impacto** parecen tener en la Diabetes en este data set.

También se observa fácilmente en este gráfico que las variables **SkinThickness**, **Insulin** y **DiabetesPedigreeFunction** tienen una **correlación muy baja** con el objetivo.

Vamos a realizar un random forest para observar los resultados, con el data set completo, en primer lugar:

Separamos las muestras de entrenamiento y test:

In [ ]:
X=data_clean.drop("Outcome",axis=1,inplace=False).copy()
y=data_clean["Outcome"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(X_train.info(),y_train.info())

**Inicialización y entrenamiento del modelo**

In [ ]:

model = DecisionTreeClassifier(random_state = 42)
model.fit(X_train, y_train)

Visualizamos el modelo

In [ ]:


fig = plt.figure(figsize=(15,15))

tree.plot_tree(model, feature_names = list(X_train.columns), class_names = ["0", "1"], filled = True)


Vemos el ajuste: 

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

Tenemos  unaccuracy del 66,23% vamos a intentar mejorarlo eliminando las variables menos correlacionadas (SkinThickness, DiabetesPedigreeFunction e Insulin)

Primero guardamos el modelo

Guardamos el modelo:

In [ ]:

dump(model, open("random_forest_def_42.sav", "wb"))

Nuevo dataset sin las variables seleccionadas:

In [ ]:
X2=data_clean.drop(["Outcome","SkinThickness","Insulin","DiabetesPedigreeFunction"],axis=1,inplace=False).copy()
X2.info()


In [ ]:
X2_train,X2_test, y2_train, y2_test= train_test_split(X2,y,train_size=0.2,random_state=42)

print(X2_train.info(),y2_train.info())

In [ ]:
model2 = DecisionTreeClassifier(random_state = 42)
model2.fit(X2_train, y2_train)

In [ ]:

fig = plt.figure(figsize=(15,15))

tree.plot_tree(model2, feature_names = list(X2_train.columns), class_names = ["0", "1"], filled = True)


In [ ]:
y2_pred=model2.predict(X2_test)
accuracy_score(y2_pred,y2_test)

Con la eliminación de estas 3 variables, hemos conseguido un modelo más sencillo y con un accuracy mejor: 67,96%

Guardamos el modelo

In [ ]:
from pickle import dump

dump(model2, open("random_forest_2_def_42.sav", "wb"))